# Project 8: Backtesting

In this project, we will build a fairly realistic backtester that uses the Barra data. The backtester will perform portfolio optimization that includes **transaction costs**, and we'll implement it with computational efficiency in mind, to allow for a reasonably fast backtest. We'll also use performance attribution to identify the major drivers of our portfolio's profit-and-loss (PnL). We will have the option to modify and customize the backtest as well.


In [2]:
import sys
!{sys.executable} -m pip install -r requirements.txt

  Using cached alphalens-0.3.2.tar.gz (18.9 MB)
  Using cached colour-0.1.5-py2.py3-none-any.whl (23 kB)
  Using cached cvxpy-1.0.3.tar.gz (880 kB)
     |████████████████████████████████| 5.0 MB 558 kB/s eta 0:00:01
  Using cached pandas-0.18.1.tar.gz (7.3 MB)
  Using cached plotly-2.2.3.tar.gz (1.1 MB)
  Using cached pyparsing-2.2.0-py2.py3-none-any.whl (56 kB)
  Using cached python_dateutil-2.6.1-py2.py3-none-any.whl (194 kB)
  Using cached pytz-2017.3-py2.py3-none-any.whl (511 kB)
  Using cached requests-2.18.4-py2.py3-none-any.whl (88 kB)
  Using cached scipy-1.0.0.tar.gz (15.2 MB)
  Using cached scikit-learn-0.19.1.tar.gz (9.5 MB)
  Using cached six-1.11.0-py2.py3-none-any.whl (10 kB)
  Using cached tables-3.3.0.tar.gz (7.0 MB)
    ERROR: Command errored out with exit status 1:
     command: /Users/jiadaihe/.pyenv/versions/3.8.6/envs/learn_tensor/bin/python3.8 -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/yh/pc7jtwg54mvchh6jp4l92jx40000gn/T/pip-inst

In [3]:
import scipy
import patsy
import pickle

import numpy as np
import pandas as pd

import scipy.sparse
import matplotlib.pyplot as plt

from statistics import median
from scipy.stats import gaussian_kde
from statsmodels.formula.api import ols
from tqdm import tqdm

## Load Data

We’ll be using the Barra dataset to get factors that can be used to predict risk. Loading and parsing the raw Barra data can be a very slow process that can significantly slow down your backtesting. For this reason, it's important to pre-process the data beforehand. Here, the Barra data has already been pre-processed and saved into pickle files. 

In the code below, we start by loading `2004` factor data from the `pandas-frames.2004.pickle` file. We also load the `2003` and `2004` covariance data from the `covaraince.2003.pickle`  and `covaraince.2004.pickle` files. The data range for the backtest can be customized. It is recommended starting with two or three years of factor data. Remember that the covariance data should include all the years that you choose for the factor data, and also one year earlier. For example, in the code below we are using  `2004` factor data, therefore, we must include `2004` in our covariance data, but also the previous year, `2003`. 

In [3]:
# load data
barra_dir = '../../data/project_8_barra/'

data = {}
for year in [2004]:
    fil = barra_dir + "pandas-frames." + str(year) + ".pickle"
    data.update(pickle.load( open( fil, "rb" ) ))
    
covariance = {}
for year in [2003, 2004]:
    fil = barra_dir + "covariance." + str(year) + ".pickle"
    covariance.update(pickle.load( open(fil, "rb" ) ))
    
daily_return = {}
for year in [2004, 2005]:
    fil = barra_dir + "price." + str(year) + ".pickle"
    daily_return.update(pickle.load( open(fil, "rb" ) ))

### Shift Daily Returns Data
In the cell below, we want to incorporate a realistic time delay that exists in live trading, we’ll use a two day delay for the `daily_return` data. That means the `daily_return` should be two days after the data in `data` and `cov_data`. Combine `daily_return` and `data` together in a dict called `frames`.


In [4]:
frames ={}
dlyreturn_n_days_delay = 2

backtest_dates = sorted(data.keys())
dlyreturn_dates = sorted(daily_return.keys())[dlyreturn_n_days_delay : len(backtest_dates)+dlyreturn_n_days_delay]

for data_date, price_date in zip(backtest_dates, dlyreturn_dates):
    frames[price_date] = data[data_date].merge(daily_return[price_date], on='Barrid')
    frames[price_date]['DlyReturnDate'] = pd.Series([price_date]*len(frames[price_date]))
    
frames['20040108'].head()

,Barrid,USFASTD_1DREVRSL,USFASTD_AERODEF,USFASTD_AIRLINES,USFASTD_ALUMSTEL,USFASTD_APPAREL,USFASTD_AUTO,USFASTD_BANKS,USFASTD_BETA,USFASTD_BEVTOB,...,ADTCA_30,IssuerMarketCap,Yield,TotalRisk,SpecRisk,HistBeta,PredBeta,DataDate,DlyReturn,DlyReturnDate
0,USA0001,-0.481,0.000,0.0,0.0,0.0,0.0,0.0,-2.158,0.0,...,NaN,5.592728e+10,0.188679,15.692850,10.050981,-0.000188,0.159701,20040106,0.0,20040108
1,USA0011,-0.595,0.000,0.0,0.0,0.0,0.0,0.0,-2.158,0.0,...,NaN,6.029930e+09,0.000000,19.050196,12.874902,0.000017,0.133397,20040106,0.0,20040108
2,USA0031,-0.109,0.000,0.0,0.0,0.0,0.0,0.0,-2.049,0.0,...,NaN,7.251836e+10,2.103004,24.037181,19.772275,0.050603,0.210419,20040106,0.0,20040108
3,USA0062,0.163,0.431,0.0,0.0,0.0,0.0,0.0,-1.997,0.0,...,NaN,2.918165e+10,2.243494,25.280406,22.709825,0.074781,0.372498,20040106,0.0,20040108
4,USA00E2,0.064,0.000,0.0,0.0,0.0,0.0,0.0,-1.955,0.0,...,NaN,5.784320e+10,2.167256,27.885397,23.513232,0.094615,0.410219,20040106,0.0,20040108


In [11]:
facret = {} # factor returns

for date in dlyreturn_dates:
    facret[date] = estimate_factor_returns(frames[date])

facret['20040108'].head()

USFASTD_1DREVRSL    0.000497
USFASTD_AERODEF     0.006654
USFASTD_AIRLINES   -0.024867
USFASTD_ALUMSTEL   -0.006506
USFASTD_APPAREL    -0.002412
dtype: float64

In [5]:
my_dates = sorted(list(map(lambda date: pd.to_datetime(date, format='%Y%m%d'), frames.keys())))
alpha_factors = ["USFASTD_1DREVRSL", "USFASTD_EARNYILD", "USFASTD_VALUE", "USFASTD_SENTMT"]
risk_aversion = 1.0e-6
previous_holdings = pd.DataFrame(data = {"Barrid" : ["USA0001"], "h.opt.previous" : np.array(0)})

### Factor Exposures and Factor Returns

$r_{i,t} = \sum_{j=1}^{k} (\beta_{i,j,t-2} \times f_{j,t})$  
where $i=1...N$ (N assets),   
and $j=1...k$ (k factors).

where $r_{i,t}$ is the return, $\beta_{i,j,t-2}$ is the factor exposure, and $f_{j,t}$ is the factor return. Since we get the factor exposures from the Barra data, and we know the returns, we use OLS to estimate the factor returns. 

In [6]:
def wins(x,a,b):
    """Winsorize, avoid extremely positive or negative values in our data."""
    return np.where(x <= a,a, np.where(x >= b, b, x))

def get_formula(factors, Y):
    L = ["0"]
    L.extend(factors)
    return Y + " ~ " + " + ".join(L)

def factors_from_names(n):
    return list(filter(lambda x: "USFASTD_" in x, n))

def estimate_factor_returns(df): 
    ## build universe based on filters 
    estu = df.loc[df.IssuerMarketCap > 1e9].copy(deep=True)
  
    ## winsorize returns for fitting 
    estu['DlyReturn'] = wins(estu['DlyReturn'], -0.25, 0.25)
  
    all_factors = factors_from_names(list(df))
    form = get_formula(all_factors, "DlyReturn")
    model = ols(form, data=estu)
    results = model.fit()
    return results.params

In [7]:
# helpers
def clean_nas(df): 
    numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
    
    for numeric_column in numeric_columns: 
        df[numeric_column] = np.nan_to_num(df[numeric_column])
    
    return df

def setdiff(temp1, temp2): 
    s = set(temp2)
    temp3 = [x for x in temp1 if x not in s]
    return temp3

def model_matrix(formula, data): 
    outcome, predictors = patsy.dmatrices(formula, data)
    return predictors

def get_universe(df):
    return df.loc[(df['IssuerMarketCap'] > 1e9) | (df['h.opt.previous'] > 0)].copy(deep=True)

### Factor covariance matrix

In [8]:
def colnames(B):
    if type(B) == patsy.design_info.DesignMatrix: 
        return B.design_info.column_names
    if type(B) == pd.core.frame.DataFrame: 
        return B.columns.tolist()
    return None

def diagonal_factor_cov(date, B):
    """
    Create the factor covariance matrix
    B : Matrix of Risk Factors
    """
    cov = covariance[date]
    factor_cov = pd.DataFrame(0, index=colnames(B), columns=colnames(B))
    for f1 in colnames(B):
        for f2 in colnames(B):
            try:
                value = cov.loc[(cov.Factor1==f1) & (cov.Factor2==f2),"VarCovar"].iloc[0]
            except:
                value = 0
            factor_cov.loc[f1,f2] = value
    return factor_cov.to_numpy()

### Transaction cost

In [9]:
def get_lambda(universe, composite_volume_column = 'ADTCA_30'):
    universe.loc[np.isnan(universe[composite_volume_column]), composite_volume_column] = 1.0e4
    universe.loc[universe[composite_volume_column] == 0, composite_volume_column] = 1.0e4 
    adv = universe[composite_volume_column]
    return 0.1 / adv

### Alpha combination
Now that you have the matrix containing the alpha factors we will combine them by adding its rows.

In [52]:
def get_B_alpha(alpha_factors, universe):
    outcome, predictors = patsy.dmatrices(get_formula(alpha_factors, "SpecRisk"), universe)
    return predictors

def get_alpha_vec(B_alpha):
    return np.sum(B_alpha, axis=1) * 1e-4

def get_alpha_vec_weighted(B_alpha, weights):
    """       
    B_alpha : Matrix of Alpha Factors, N * # alpha factors
    weights: vector, # alpha factors * 1
    Return an alpha vecrtor, N*1
    """
    return np.matmul(B_alpha, weights) * 1e-4

from bisect import bisect_left 

def alpha_weights(alpha_factors, date, window_length=15):
    '''Return a weight vector, # alpha factors * 1'''
    today = bisect_left(my_dates, pd.to_datetime(date, format='%Y%m%d')) 
    if today <= window_length:
        return np.array([1/len(alpha_factors)] * len(alpha_factors))
    
    start = today - dlyreturn_n_days_delay - window_length if today-dlyreturn_n_days_delay-window_length > 0 else 0
    alpha_returns = []
    for i in range(start, today - dlyreturn_n_days_delay + 1):
        dt = my_dates[i].strftime('%Y%m%d')
        results = facret[dt]
        alpha_returns.append(results[alpha_factors])
    
    sharps = np.mean(alpha_returns, axis=0)/np.std(alpha_returns, axis=0)
    sharps = np.nan_to_num(sharps)
    sharps = np.maximum(sharps, 0)
    return np.array(sharps/np.sum(sharps))

### Optimize

In [26]:
def get_obj_func(h0, risk_aversion, Q, specVar, alpha_vec, Lambda): 
    '''h0: vector, N*1
       risk_aversion: scalar
       Q: matrix, K*N
       specVar: vector, N*1
       alpha_vec: vector, N*1
       Lambda: vector, N*1
    '''
    def obj_func(h):
        f = 0.5 * risk_aversion * np.sum(np.matmul(Q, h)**2) + \
            0.5 * risk_aversion * np.dot(h**2, specVar) - \
            np.dot(alpha_vec, h) + np.dot((h-h0)**2, Lambda)
        return np.asarray(f)
    
    return obj_func

def get_grad_func(h0, risk_aversion, Q, QT, specVar, alpha_vec, Lambda):
    def grad_func(h):
        f_prime = risk_aversion * np.matmul(QT, (np.matmul(Q, h))) + \
                  risk_aversion * h * specVar - \
                  alpha_vec + 2 * (h-h0) * Lambda
        return np.asarray(f_prime)
    
    return grad_func

def get_h_star(risk_aversion, Q, QT, specVar, alpha_vec, h0, Lambda):
    """
    Optimize the objective function
    Return Numpy ndarray -- optimized holdings
    """
    obj_func = get_obj_func(h0, risk_aversion, Q, specVar, alpha_vec, Lambda)
    grad_func = get_grad_func(h0, risk_aversion, Q, QT, specVar, alpha_vec, Lambda)
    optimizer_result = scipy.optimize.fmin_l_bfgs_b(obj_func, h0, fprime=grad_func)
    return optimizer_result[0]

### Risk Exposures, Alpha Exposures

We can also use `h_star` to calculate our portfolio's risk and alpha exposures.

In [15]:
def get_risk_exposures(B, BT, h_star):
    return pd.Series(np.matmul(BT, h_star), index = colnames(B))

def get_portfolio_alpha_exposure(B_alpha, h_star):
    return pd.Series(np.matmul(B_alpha.transpose(), h_star), index = colnames(B_alpha))

### Transaction Costs 

We can also use `h_star` to calculate our total transaction costs:
$$
\mbox{tcost} = \sum_i^{N} \lambda_{i} (h_{i,t} - h_{i,t-1})^2
$$


In [27]:
def get_total_transaction_costs(h0, h_star, Lambda):
    return np.sum( (h_star - h0)**2 * Lambda)

## Run the backtest

$$
f(\mathbf{h}) = \frac{1}{2}\kappa \mathbf{h}_t^T\textbf{BFB}^T\mathbf{h}_t + \frac{1}{2} \kappa \mathbf{h}_t^T \mathbf{S} \mathbf{h}_t - \mathbf{\alpha}^T \mathbf{h}_t + (\mathbf{h}_{t} - \mathbf{h}_{t-1})^T \mathbf{\Lambda} (\mathbf{h}_{t} - \mathbf{h}_{t-1})
$$
where $\textbf{BFB}^T=\textbf{Q}^T\textbf{Q}$

In [61]:
def form_optimal_portfolio(df, previous, risk_aversion, weight_alpha=False):
    df = df.merge(previous, how = 'left', on = 'Barrid') # apprend previous holdings to the dataframe, column name is h.opt.previous
    df = clean_nas(df) # replace NaN with zero and infinity with large finite numbers
    df.loc[df['SpecRisk'] == 0]['SpecRisk'] = median(df['SpecRisk']) 
  
    universe = get_universe(df) # market cap > 1 billion or previously holding it
    date = str(int(universe['DataDate'][1])) # the date of factor exposures (betas) in Barra data 

    all_factors = factors_from_names(list(universe))
    risk_factors = setdiff(all_factors, alpha_factors)
  
    h0 = universe['h.opt.previous']
  
    B = model_matrix(get_formula(risk_factors, "SpecRisk"), universe) # B at data_date
    BT = B.transpose()
  
    specVar = (0.01 * universe['SpecRisk']) ** 2 # specific variance at data_date
    Fvar = diagonal_factor_cov(date, B) # risk factor covariance matrix at data_date
    
    Lambda = get_lambda(universe)
    B_alpha = get_B_alpha(alpha_factors, universe)
    alpha_vec = get_alpha_vec(B_alpha)
    
    if weight_alpha:
        weights = alpha_weights(alpha_factors, date)
        alpha_vec = get_alpha_vec_weighted(B_alpha, weights)
    
    Q = np.matmul(scipy.linalg.sqrtm(Fvar), BT)
    QT = Q.transpose()
    
    h_star = get_h_star(risk_aversion, Q, QT, specVar, alpha_vec, h0, Lambda)
    opt_portfolio = pd.DataFrame(data = {"Barrid" : universe['Barrid'], "h.opt" : h_star})
    
    risk_exposures = get_risk_exposures(B, BT, h_star)
    portfolio_alpha_exposure = get_portfolio_alpha_exposure(B_alpha, h_star)
    total_transaction_costs = get_total_transaction_costs(h0, h_star, Lambda)
  
    return {
        "opt.portfolio" : opt_portfolio, 
        "risk.exposures" : risk_exposures, 
        "alpha.exposures" : portfolio_alpha_exposure,
        "total.cost" : total_transaction_costs}

def build_tradelist(prev_holdings, opt_result):
    tmp = prev_holdings.merge(opt_result['opt.portfolio'], how='outer', on = 'Barrid')
    tmp['h.opt.previous'] = np.nan_to_num(tmp['h.opt.previous'])
    tmp['h.opt'] = np.nan_to_num(tmp['h.opt'])
    return tmp

def convert_to_previous(result): 
    prev = result['opt.portfolio']
    prev = prev.rename(index=str, columns={"h.opt": "h.opt.previous"}, copy=True, inplace=False)
    return prev

In [62]:
trades = {}
port = {}

for dt in tqdm(my_dates[:5], desc='Optimizing Portfolio', unit='day'):
    date = dt.strftime('%Y%m%d')

    result = form_optimal_portfolio(frames[date], previous_holdings, risk_aversion)
    trades[date] = build_tradelist(previous_holdings, result)
    port[date] = result
    previous_holdings = convert_to_previous(result)

Optimizing Portfolio:   0%|          | 0/5 [00:00<?, ?day/s]

[-1.5200e-05 -6.9200e-05 -9.0750e-06 ...  4.7500e-07  4.1975e-05
  9.8250e-06]


Optimizing Portfolio:  20%|██        | 1/5 [00:38<02:32, 38.20s/day]

[-8.1000e-06 -8.2150e-05 -3.2425e-05 ...  3.4525e-05 -8.7500e-06
  1.4100e-05]


Optimizing Portfolio:  40%|████      | 2/5 [01:15<01:53, 37.88s/day]

...

Optimizing Portfolio:  60%|██████    | 3/5 [01:52<01:14, 37.38s/day]

[-8.81462569e-05 -1.20896907e-04 -7.35331918e-06 ... -6.29743424e-05
  1.68873398e-04 -6.37774347e-06]


Optimizing Portfolio:  80%|████████  | 4/5 [02:30<00:37, 37.72s/day]

[-5.26022567e-05 -1.08979133e-04 -3.54104118e-05 ... -1.63019099e-05
  6.44950574e-05 -5.13865069e-05]


Optimizing Portfolio: 100%|██████████| 5/5 [03:07<00:00, 37.48s/day]


### Profit-and-Loss (PnL) attribution

In [59]:
## assumes v, w are pandas Series 
def partial_dot_product(v, w):
    common = v.index.intersection(w.index)
    return np.sum(v[common] * w[common])

def build_pnl_attribution(): 

    df = pd.DataFrame(index = my_dates)
    
    for dt in my_dates[:2]:
        date = dt.strftime('%Y%m%d')
        
        p = port[date]
        fr = facret[date] # factor returns
        
        mf = p['opt.portfolio'].merge(frames[date], how = 'left', on = "Barrid")
        
        mf['DlyReturn'] = wins(mf['DlyReturn'], -0.5, 0.5)
        df.at[dt,"daily.pnl"] = np.sum(mf['h.opt'] * mf['DlyReturn'])

        # TODO: Implement
        df.at[dt,"attribution.alpha.pnl"] = partial_dot_product(fr, p["alpha.exposures"])
        df.at[dt,"attribution.risk.pnl"] = partial_dot_product(fr, p["risk.exposures"] )
        df.at[dt,"attribution.cost"] = p["total.cost"]
        
    return df

In [63]:
build_pnl_attribution()

,daily.pnl,attribution.alpha.pnl,attribution.risk.pnl,attribution.cost
2004-01-06,238.407678,1489.421637,0.006977,6.050174
2004-01-07,-600.189818,308.273480,0.021572,3.561889
2004-01-08,NaN,NaN,NaN,NaN
2004-01-09,NaN,NaN,NaN,NaN
2004-01-12,NaN,NaN,NaN,NaN
2004-01-13,NaN,NaN,NaN,NaN
2004-01-14,NaN,NaN,NaN,NaN
2004-01-15,NaN,NaN,NaN,NaN
2004-01-16,NaN,NaN,NaN,NaN
2004-01-20,NaN,NaN,NaN,NaN


In [38]:
df = frames['20040106'].merge(previous_holdings, how = 'left', on = 'Barrid') 
df = clean_nas(df) # replace NaN with zero and infinity with large finite numbers
df.loc[df['SpecRisk'] == 0]['SpecRisk'] = median(df['SpecRisk']) 

universe = get_universe(df)
B_alpha = get_B_alpha(alpha_factors, universe)

In [64]:
B_alpha.shape

(2267, 4)